In [61]:
import numpy as np, pandas as pd
import json, os, datetime, time
import talib as ta
import matplotlib.pyplot as plt, plotly
import requests
import MetaTrader5 as mt5
from beeprint import pp
import bios




login =  99999
password =  "9999"
server =  "9999"

def connect(login=login, password=password, server=server):
    if not mt5.initialize(login=login, password=password,server=server):
        print("initialize() failed, error code =",mt5.last_error())

connect(login, password, server)

# get list_symbol

In [7]:
list_field = []
for symbol in mt5.symbols_get('*'):
    main_field = symbol.path.split('\\')[0]
    if main_field not in list_field:
        print(main_field)
        list_field.append(main_field)

Stocks
Forex
Forex Major
Crypto Currency
CFDs
Oil
Commodities
Nikkei
Silver
Gold


## import monthly df

In [71]:
def get_df_month(symbol, year, month, add_date=False):
    init_date = datetime.datetime(year, month, 1)
    next_month = month % 12 + 1
    next_year = year + month // 12
    end_date = datetime.datetime(next_year, next_month, 1)
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M1, init_date, end_date)
    if len(rates) > 0:
        df = pd.DataFrame(rates)
        df = df.drop('real_volume',axis=1)
        if add_date:
            df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')
        return df
    else:
        return pd.DataFrame()
df = get_df_month('AUDUSD', 2020, 5, True)
df

,time,open,high,low,close,tick_volume,spread,date
0,1588284000,0.65206,0.65231,0.65203,0.65219,43,15,2020-04-30 22:00:00
1,1588284060,0.65219,0.65230,0.65219,0.65228,26,14,2020-04-30 22:01:00
2,1588284120,0.65229,0.65236,0.65228,0.65233,28,15,2020-04-30 22:02:00
3,1588284180,0.65233,0.65233,0.65223,0.65226,23,15,2020-04-30 22:03:00
4,1588284240,0.65226,0.65233,0.65221,0.65229,37,15,2020-04-30 22:04:00
...,...,...,...,...,...,...,...,...
30347,1590796500,0.66672,0.66683,0.66669,0.66679,27,18,2020-05-29 23:55:00
30348,1590796560,0.66681,0.66687,0.66652,0.66657,46,19,2020-05-29 23:56:00
30349,1590796620,0.66657,0.66675,0.66630,0.66632,58,55,2020-05-29 23:57:00
30350,1590796680,0.66632,0.66645,0.66618,0.66645,18,54,2020-05-29 23:58:00


## concatenate df

In [48]:
def concat_successive(df1,df2):
    end_time = df1.iloc[-1].time
    start_time = df2.iloc[0].time
    if end_time == start_time:
        return pd.concat([df1.iloc[:-1],df2],axis=0).reset_index(drop=True)
    else:
        return pd.concat([df1,df2],axis=0).reset_index(drop=True)

In [56]:
concat_successive(get_df_month(list_symbol[0], 2020, 6, True),get_df_month(list_symbol[0], 2020, 6+1, True))

,time,open,high,low,close,tick_volume,spread,date
0,1590969600,0.66565,0.66569,0.66551,0.66551,4,60,2020-06-01 00:00:00
1,1590969660,0.66567,0.66567,0.66567,0.66567,1,61,2020-06-01 00:01:00
2,1590969720,0.66567,0.66567,0.66554,0.66567,3,61,2020-06-01 00:02:00
3,1590969780,0.66567,0.66575,0.66567,0.66575,2,43,2020-06-01 00:03:00
4,1590969840,0.66561,0.66580,0.66561,0.66580,3,43,2020-06-01 00:04:00
...,...,...,...,...,...,...,...,...
64612,1596232560,0.71409,0.71409,0.71401,0.71409,31,14,2020-07-31 21:56:00
64613,1596232620,0.71409,0.71417,0.71408,0.71410,19,14,2020-07-31 21:57:00
64614,1596232680,0.71410,0.71443,0.71410,0.71442,38,14,2020-07-31 21:58:00
64615,1596232740,0.71443,0.71453,0.71433,0.71449,104,14,2020-07-31 21:59:00


## get list symbol

In [86]:
field = 'Forex'
broker = 'VantageFX'
list_symbol = []
for symbol in mt5.symbols_get('*'):
    if symbol.path.split('\\')[0] == field:
        list_symbol.append(symbol.name)

## loop on symbols

In [ ]:
def loop(list_symbol, field, broker, debug=False):
    if field not in os.listdir('D://Trading/Data'):
        os.mkdir(f'D://Trading/Data/{field}')
    if broker not in os.listdir(f'D://Trading/Data/{field}'):
        os.mkdir(f'D://Trading/Data/{field}/{broker}')
        os.mkdir(f'D://Trading/Data/{field}/{broker}/Price')
    for symbol in list_symbol:
        print(f'{datetime.datetime.now()} - importing data of {symbol}')
        mt5.symbol_select(symbol, True)
        keep_downloading = True
        for year in range(datetime.datetime.now().year,1999,-1):
            #print(f'{datetime.datetime.now()} - year {year}')
            for month in range(12,0,-1):                
                if (year == datetime.datetime.now().year and month < datetime.datetime.now().month) or year < datetime.datetime.now().year:
                    if keep_downloading:
                        
                        date = datetime.datetime(year, month, 1)
                        year_str = date_to_ymd_str(date)[0]
                        month_str = date_to_ymd_str(date)[1]
                        allowDL = symbol not in os.listdir(f'D://Trading/Data/{field}/{broker}/Price/')
                        if not allowDL:
                            list_year_str = os.listdir(f'D://Trading/Data/{field}/{broker}/Price/{symbol}/1min')
                            allowDL = year_str not in list_year_str
                            if not allowDL:                                
                                filename = f'{year_str}-{month_str}.csv.gz'
                                list_filename = os.listdir(f'D://Trading/Data/{field}/{broker}/Price/{symbol}/1min/{year_str}')
                                allowDL = filename not in list_filename
                        
                        if allowDL:
                            
                            if debug:
                                print(f'{datetime.datetime.now()} - year = {year}  month = {month}')
                                
                            date = datetime.datetime(year, month, 1)
                            path = f'D://Trading/Data/{field}/{broker}/Price/:symbol/1min/:year/:year-:month.csv.gz'

                            params = {
                                "symbol": symbol,
                                "year": date_to_ymd_str(date)[0],
                                "month": date_to_ymd_str(date)[1]
                            }
                            for key, item in params.items():
                                path = path.replace(':'+key, item)

                            df = get_df_month(symbol, year, month, False)

                            if len(df) > 0 :       
                                folders_to_create = path[4:].split('/')[:-1]
                                tmp_path = 'D://'
                                for folder in folders_to_create:
                                    if folder not in os.listdir(tmp_path):
                                        os.mkdir(tmp_path+folder)
                                    tmp_path += folder + '/'

                                df.to_csv(path,index=False)

                            else:
                                print(f'{symbol} {year}-{month} is empty: ',df)
                                keep_downloading = False
                    
        
loop(list_symbol, field, broker, False)

2021-03-24 18:43:03.542709 - importing data of AUDCAD
AUDCAD 2007-4 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.566623 - importing data of AUDCHF
AUDCHF 2007-4 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.588593 - importing data of AUDCNH
2021-03-24 18:43:03.614495 - importing data of AUDJPY
AUDJPY 2006-10 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.637461 - importing data of AUDNZD
AUDNZD 2007-4 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.660399 - importing data of AUDSGD
2021-03-24 18:43:03.686304 - importing data of CADCHF
CADCHF 2008-12 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.930457 - importing data of CADJPY
2021-03-24 18:43:03.956388 - importing data of CHFJPY
CHFJPY 2006-10 is empty:  Empty DataFrame
Columns: []
Index: []
2021-03-24 18:43:03.975340 - importing data of CHFSGD
2021-03-24 18:43:04.001267 - importing data of EURAUD
2021-03-24 18:43:04.02719